In [ ]:
!pip install flwr["simulation"]==0.19

In [2]:
from collections import OrderedDict
import importlib
import random
from typing import Dict, List, Optional, Tuple

import flwr as fl
import numpy as np
import torch
import torch.nn as nn
import torchvision
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split, Subset
from torchvision import datasets

importlib.import_module(".common", "flwr.dataset.utils")

<module 'flwr.dataset.utils.common' from '/usr/local/lib/python3.7/dist-packages/flwr/dataset/utils/common.py'>

In [3]:
DATASET = "CIFAR10"  # possible values: "CIFAR10" or "CIFAR100"
NUM_CLIENTS = 50
NUM_ROUNDS = 10
LOCAL_EPOCHS = 2
BATCH_SIZE = 128
LR = 0.1
MILESTONES = [60, 120, 160]
LR_DECAY = 0.2
W_DECAY = 5e-04
FRACTION_FIT = 1/3
FRACTION_EVAL = 0
SEED = 0
# Set the start method for multiprocessing in case Python version is under 3.8.1
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Training on {DEVICE}")

Training on cuda


In [4]:
def set_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

In [5]:
def load_datasets(num_clients: int, dataset: str = "CIFAR10"):
    if dataset not in ["CIFAR10", "CIFAR100"]:
        raise ValueError("Unknown dataset! Admissible values are: 'CIFAR10' or 'CIFAR100'.")
    # Download and transform CIFAR dataset (train and test)
    augmentation = [transforms.Pad(4), transforms.RandomHorizontalFlip(), transforms.RandomCrop(32)]
    transform = [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
    trainset = getattr(datasets, dataset)("./dataset", train=True, download=True, transform=transforms.Compose([*augmentation, *transform]))
    testset = getattr(datasets, dataset)("./dataset", train=False, download=True, transform=transforms.Compose(transform))

    # Split training set into `num_clients` partitions to simulate different local datasets
    targets = np.array(trainset.targets)
    idxs = np.array(range(len(targets)))
    dataset = [idxs, targets]
    train_partitions, _ = fl.dataset.utils.common.create_lda_partitions(
        dataset, num_partitions=NUM_CLIENTS, concentration=1/3, accept_imbalanced=False
    )

    # Split each partition into train/val and create DataLoader
    trainloaders = []
    valloaders = []
    for ds in map(lambda p: Subset(trainset, p[0]), train_partitions):
        len_val = len(ds) // 10  # 10 % validation set
        len_train = len(ds) - len_val
        lengths = [len_train, len_val]
        ds_train, ds_val = random_split(ds, lengths, torch.Generator())
        trainloaders.append(DataLoader(ds_train, batch_size=BATCH_SIZE, shuffle=True))
        valloaders.append(DataLoader(ds_val, batch_size=BATCH_SIZE))
    testloader = DataLoader(testset, batch_size=BATCH_SIZE)
    return trainloaders, valloaders, testloader 

In [6]:
class Net(nn.Module):
    def __init__(self, num_classes: int) -> None:
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, num_classes)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


def get_parameters(net) -> List[np.ndarray]:
    return [val.cpu().numpy() for _, val in net.state_dict().items()]


def set_parameters(net, parameters: List[np.ndarray]):
    params_dict = zip(net.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
    net.load_state_dict(state_dict, strict=True)


def train(net, trainloader, epochs: int, lr: float = 0.1):
    """Train the network on the training set."""
    criterion = torch.nn.CrossEntropyLoss().to(DEVICE)
    optimizer = torch.optim.SGD(net.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4)
    net.train()
    for epoch in range(epochs):
        correct, total, epoch_loss = 0, 0, 0.0
        for images, labels in trainloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = net(images)
            loss = criterion(net(images), labels)
            loss.backward()
            optimizer.step()
            # Metrics
            epoch_loss += loss.item() * labels.size(0)
            total += labels.size(0)
            correct += (torch.max(outputs.data, 1)[1] == labels).sum().item()
        epoch_loss /= len(trainloader.dataset)
        epoch_acc = correct / total
        print(f"Epoch {epoch+1}: train loss {epoch_loss}, accuracy {epoch_acc}")


def test(net: nn.Module, testloader: DataLoader, return_dict = None):
    """Evaluate the network on the entire test set."""
    criterion = torch.nn.CrossEntropyLoss().to(DEVICE)
    correct, total, loss = 0, 0, 0.0
    net.eval()
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = net(images)
            loss += criterion(outputs, labels).item() * labels.size(0)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    loss /= len(testloader.dataset)
    accuracy = correct / total
    return loss, accuracy

In [7]:
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, cid, net, trainloader, valloader):
        self.cid = cid
        self.net = net.to(DEVICE)
        self.trainloader = trainloader
        self.valloader = valloader

    def get_parameters(self):
        print(f"[Client {self.cid}] get_parameters")
        return get_parameters(self.net)

    def fit(self, parameters, config):
        # Read values from config
        current_round = config["current_round"]
        local_epochs = config["local_epochs"]
        lr = config["lr"]
        # Use values provided by the config
        print(f"[Client {self.cid}, round {current_round}] fit, config: {config}")
        set_parameters(self.net, parameters)
        train(self.net, self.trainloader, local_epochs)
        return get_parameters(self.net), len(self.trainloader), {}

    def evaluate(self, parameters, config):
        print(f"[Client {self.cid}] evaluate, config: {config}")
        set_parameters(self.net, parameters)
        loss, accuracy = test(self.net, self.valloader)
        return float(loss), len(self.valloader), {"accuracy": float(accuracy)}

In [8]:
set_seed(SEED)

# Load data
trainloaders, valloaders, testloader = load_datasets(NUM_CLIENTS, DATASET)
NUM_CLASSES = len(np.unique(testloader.dataset.targets))

# Create an instance of the model and get the parameters
net = Net(NUM_CLASSES).to(DEVICE)
params = get_parameters(net)
print(sum(p.numel() for p in net.parameters() if p.requires_grad))

Files already downloaded and verified
Files already downloaded and verified
62006


In [9]:
def client_fn(cid) -> FlowerClient:
    net = Net(NUM_CLASSES).to(DEVICE)
    trainloader = trainloaders[int(cid)]
    valloader = valloaders[int(cid)]
    return FlowerClient(cid, net, trainloader, valloader)

def evaluate(
    weights: fl.common.Weights,
) -> Optional[Tuple[float, Dict[str, fl.common.Scalar]]]:
    net = Net(NUM_CLASSES).to(DEVICE)
    set_parameters(net, weights)  # Update model with the latest parameters
    loss, accuracy = test(net, testloader)
    return loss, {"accuracy": accuracy}

def fit_config(rnd: int):
    lr = LR
    if MILESTONES is not None and LR_DECAY is not None:
        lr *= LR_DECAY ** sum([1 if rnd >= e else 0 for e in MILESTONES])
    config = {
        "current_round": rnd,
        "local_epochs": 1 if rnd < 2 else LOCAL_EPOCHS,
        "lr": lr
    }
    return config

strategy = fl.server.strategy.FedAvg(
    fraction_fit=FRACTION_FIT,
    fraction_eval=FRACTION_EVAL,
    min_fit_clients=int(FRACTION_FIT * NUM_CLIENTS),
    min_eval_clients=int(FRACTION_EVAL * NUM_CLIENTS),
    min_available_clients=NUM_CLIENTS,
    initial_parameters=fl.common.weights_to_parameters(get_parameters(net)),
    on_fit_config_fn=fit_config,
    eval_fn=evaluate,
)

fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    num_rounds=10,
    strategy=strategy,
    client_resources={"num_cpus": 1, "num_gpus": 1},
    ray_init_args={"local_mode": True, "include_dashboard": False}
)

INFO flower 2022-06-01 14:17:03,529 | app.py:158 | Ray initialized with resources: {'accelerator_type:T4': 1.0, 'node:172.28.0.2': 1.0, 'memory': 5701926912.0, 'GPU': 1.0, 'CPU': 2.0, 'object_store_memory': 2850963456.0}
INFO flower 2022-06-01 14:17:03,535 | app.py:174 | Starting Flower simulation running: Config(num_rounds=10, round_timeout=None)
INFO flower 2022-06-01 14:17:03,539 | server.py:84 | Initializing global parameters
INFO flower 2022-06-01 14:17:03,543 | server.py:252 | Using initial parameters provided by strategy
INFO flower 2022-06-01 14:17:03,547 | server.py:86 | Evaluating initial parameters
INFO flower 2022-06-01 14:17:06,235 | server.py:93 | initial parameters (loss, other metrics): 2.3050458755493164, {'accuracy': 0.1011}
INFO flower 2022-06-01 14:17:06,237 | server.py:99 | FL starting
DEBUG flower 2022-06-01 14:17:06,241 | server.py:207 | fit_round: strategy sampled 16 clients (out of 50)
2022-06-01 14:17:13,484	WARNING worker.py:528 -- `ray.get_gpu_ids()` will al

:task_name:launch_and_fit


:task_name:launch_and_fit
:task_name:launch_and_fit
:task_name:launch_and_fit
:task_name:launch_and_fit
:task_name:launch_and_fit
:task_name:launch_and_fit
:task_name:launch_and_fit
:task_name:launch_and_fit
:task_name:launch_and_fit


:task_name:launch_and_fit
:task_name:launch_and_fit
:task_name:launch_and_fit
:task_name:launch_and_fit
:task_name:launch_and_fit
:task_name:launch_and_fit
:task_name:launch_and_fit
:task_name:launch_and_fit
[Client 34, round 1] fit, config: {'current_round': 1, 'local_epochs': 1, 'lr': 0.1}
[Client 36, round 1] fit, config: {'current_round': 1, 'local_epochs': 1, 'lr': 0.1}:task_name:launch_and_fit

[Client 20, round 1] fit, config: {'current_round': 1, 'local_epochs': 1, 'lr': 0.1}
[Client 15, round 1] fit, config: {'current_round': 1, 'local_epochs': 1, 'lr': 0.1}
[Client 43, round 1] fit, config: {'current_round': 1, 'local_epochs': 1, 'lr': 0.1}
[Client 5, round 1] fit, config: {'current_round': 1, 'local_epochs': 1, 'lr': 0.1}
[Client 28, round 1] fit, config: {'current_round': 1, 'local_epochs': 1, 'lr': 0.1}
[Client 32, round 1] fit, config: {'current_round': 1, 'local_epochs': 1, 'lr': 0.1}
[Client 9, round 1] fit, config: {'current_round': 1, 'local_epochs': 1, 'lr': 0.1}
[Cl

:task_name:launch_and_fit
:task_name:launch_and_fit


...

Epoch 2: train loss 1.3059204117457073, accuracy 0.5366666666666666
Epoch 2: train loss 1.5488160806232028, accuracy 0.4888888888888889
Epoch 2: train loss 1.4163131141662597, accuracy 0.51
Epoch 2: train loss 1.7747280597686768, accuracy 0.2577777777777778
Epoch 1: train loss 1.7117728763156468, accuracy 0.4922222222222222
Epoch 1: train loss 1.6648481803470188, accuracy 0.43444444444444447
Epoch 1: train loss 1.996812441613939, accuracy 0.2877777777777778
Epoch 1: train loss 2.105550931294759, accuracy 0.24555555555555555
Epoch 1: train loss 1.5759341740608215, accuracy 0.5533333333333333
Epoch 1: train loss 1.9616680632697212, accuracy 0.2577777777777778
Epoch 2: train loss 1.8000498708089192, accuracy 0.31333333333333335
Epoch 2: train loss 1.2336133522457546, accuracy 0.5955555555555555
Epoch 2: train loss 0.9479498631424375, accuracy 0.56
Epoch 2: train loss 1.1510335932837592, accuracy 0.6277777777777778
Epoch 2: train loss 1.7253431865904065, accuracy 0.30666666666666664


DEBUG flower 2022-06-01 14:24:47,726 | server.py:220 | fit_round received 16 results and 0 failures


Epoch 2: train loss 1.880741523636712, accuracy 0.31333333333333335


INFO flower 2022-06-01 14:24:50,061 | server.py:120 | fit progress: (10, 2.2211602916717528, {'accuracy': 0.1534}, 463.81957093699987)
INFO flower 2022-06-01 14:24:50,063 | server.py:155 | evaluate_round: no clients selected, cancel
INFO flower 2022-06-01 14:24:50,068 | server.py:138 | FL finished in 463.82686970299994
INFO flower 2022-06-01 14:24:50,073 | app.py:178 | app_fit: losses_distributed []
INFO flower 2022-06-01 14:24:50,078 | app.py:179 | app_fit: metrics_distributed {}
INFO flower 2022-06-01 14:24:50,081 | app.py:180 | app_fit: losses_centralized [(0, 2.3050458755493164), (1, 2.331439879989624), (2, 2.323782335662842), (3, 2.3351965183258057), (4, 2.356150206756592), (5, 2.3107319602966307), (6, 2.3805796180725096), (7, 2.20575557975769), (8, 2.2400429752349855), (9, 2.2648050483703615), (10, 2.2211602916717528)]
INFO flower 2022-06-01 14:24:50,083 | app.py:181 | app_fit: metrics_centralized {'accuracy': [(0, 0.1011), (1, 0.0931), (2, 0.1), (3, 0.1435), (4, 0.1), (5, 0.1586

History (loss, centralized):
	round 0: 2.3050458755493164
	round 1: 2.331439879989624
	round 2: 2.323782335662842
	round 3: 2.3351965183258057
	round 4: 2.356150206756592
	round 5: 2.3107319602966307
	round 6: 2.3805796180725096
	round 7: 2.20575557975769
	round 8: 2.2400429752349855
	round 9: 2.2648050483703615
	round 10: 2.2211602916717528
History (metrics, centralized):
{'accuracy': [(0, 0.1011), (1, 0.0931), (2, 0.1), (3, 0.1435), (4, 0.1), (5, 0.1586), (6, 0.1), (7, 0.1382), (8, 0.1526), (9, 0.1154), (10, 0.1534)]}